In [ ]:
!pip install plotly yfinance  # Instala las librerías necesarias (solo la primera vez)

# PROYECTO DE MACHINE LEARNING: PRECIO DEL CAFÉ


import pandas as pd                # Para manejo de datos (tablas)
import numpy as np                 # Para cálculos numéricos
import plotly.graph_objects as go  # Para gráficos interactivos
from sklearn.ensemble import RandomForestRegressor  # Modelo de regresión
from sklearn.svm import SVC                          # Modelo de clasificación (sube/baja)
from sklearn.preprocessing import StandardScaler     # Escala datos para el SVM
from sklearn.metrics import mean_absolute_error, accuracy_score  # Métricas del modelo
import yfinance as yf             # Librería para descargar datos financieros reales


# DATOS DEL CAFÉ
class DatosCafe:
    def __init__(self):
        self.df = None  # Aquí se guardará el DataFrame de los datos

    def bajar_datos(self):
        """Descarga los datos históricos del café desde Yahoo Finance"""
        print("Bajando datos reales del cafe (KC=F)...")
        try:
            # Descarga el precio de cierre del futuro del café (KC=F)
            data = yf.download('KC=F', period='2y', progress=False)
            self.df = data[['Close']].copy()  # Toma solo la columna "Close"
            self.df.columns = ['cafe']        # Renombra a 'cafe'
            self.df = self.df.dropna()        # Elimina filas vacías
        except:
            # Si falla la descarga, genera datos simulados
            print("Error, uso datos falsos")
            fechas = pd.date_range(end='2025-11-06', periods=500)
            precios = 380 + np.cumsum(np.random.normal(0, 8, 500))
            self.df = pd.DataFrame({'cafe': precios}, index=fechas)

    def hacer_features(self):
        """Crea columnas adicionales (variables) que ayudan al modelo"""
        print("Creando features...")
        self.df['ayer'] = self.df['cafe'].shift(1)         # Precio del día anterior
        self.df['ma5'] = self.df['cafe'].rolling(5).mean()  # Promedio móvil 5 días
        self.df['ma20'] = self.df['cafe'].rolling(20).mean()# Promedio móvil 20 días

        # Cálculo de volatilidad y RSI
        self.df['retorno'] = self.df['cafe'].pct_change()   # Variación porcentual diaria
        self.df['volatilidad'] = self.df['retorno'].rolling(20).std()  # Volatilidad 20 días

        # RSI (indicador técnico de fuerza relativa)
        delta = self.df['cafe'].diff()
        up = delta.where(delta > 0, 0).rolling(14).mean()
        down = -delta.where(delta < 0, 0).rolling(14).mean()
        self.df['rsi'] = 100 - (100 / (1 + up / down))

        self.df = self.df.dropna()  # Elimina valores vacíos
        print(f"Listo: {len(self.df)} filas")  # Muestra cantidad final de datos


# MODELO DE REGRESIÓN (PREDECIR PRECIO)
class ModeloPrecio:
    def __init__(self):
        # Crea un modelo de Random Forest (bosque aleatorio)
        self.modelo = RandomForestRegressor(n_estimators=50, random_state=42)
        self.features = ['ayer', 'ma5', 'ma20', 'volatilidad', 'rsi']  # Variables a usar

    def entrenar(self, X, y):
        """Entrena el modelo con los datos históricos"""
        print("Entrenando Random Forest...")
        self.modelo.fit(X, y)

    def predecir(self, X):
        """Hace predicciones con los datos nuevos"""
        return self.modelo.predict(X)


# MODELO DE CLASIFICACIÓN (SUBE/BAJA)
class ModeloSubeBaja:
    def __init__(self):
        self.modelo = SVC(kernel='rbf', C=1.0)  # Clasificador tipo SVM
        self.scaler = StandardScaler()          # Normaliza los datos

    def entrenar(self, X, y):
        """Entrena el modelo SVM para saber si el café sube o baja"""
        print("Entrenando SVM...")
        X_scaled = self.scaler.fit_transform(X)  # Escala los datos
        self.modelo.fit(X_scaled, y)

    def predecir(self, X):
        """Predice si el precio sube (1) o baja (0)"""
        X_scaled = self.scaler.transform(X)
        return self.modelo.predict(X_scaled)


# FUNCION PARA DIVIDIR DATOS EN TRAIN Y TEST
def dividir_datos(df, test_size=0.2):
    """Separa los datos en entrenamiento y prueba"""
    split = int(len(df) * (1 - test_size))
    return df.iloc[:split], df.iloc[split:]


# PROGRAMA PRINCIPAL
if __name__ == "__main__":

    # 1. DESCARGAR Y PREPARAR DATOS
    datos = DatosCafe()
    datos.bajar_datos()
    datos.hacer_features()

    # 2. CREAR VARIABLES OBJETIVO
    datos.df['precio_mañana'] = datos.df['cafe'].shift(-1)               # Valor del siguiente día
    datos.df['sube'] = (datos.df['precio_mañana'] > datos.df['cafe']).astype(int)  # Si sube = 1, baja = 0
    datos.df = datos.df[:-1]  # Elimina la última fila (no tiene valor de "mañana")

    # 3. DIVIDIR DATOS EN TRAIN Y TEST
    train_df, test_df = dividir_datos(datos.df)

    # Variables predictoras (X) y objetivo (y)
    X_train = train_df[['ayer', 'ma5', 'ma20', 'volatilidad', 'rsi']].values
    X_test  = test_df[['ayer', 'ma5', 'ma20', 'volatilidad', 'rsi']].values

    y_precio_train = train_df['precio_mañana'].values
    y_precio_test  = test_df['precio_mañana'].values

    y_clase_train = train_df['sube'].values
    y_clase_test  = test_df['sube'].values

    # 4. ENTRENAR MODELO DE REGRESIÓN
    modelo_p = ModeloPrecio()
    modelo_p.entrenar(X_train, y_precio_train)
    pred_test_p = modelo_p.predecir(X_test)

    # Calcular error promedio (MAE)
    mae = mean_absolute_error(y_precio_test, pred_test_p)
    print(f"Error en test: {mae:.1f} USD/Lbs")

    # 5. ENTRENAR MODELO DE CLASIFICACIÓN
    modelo_c = ModeloSubeBaja()
    modelo_c.entrenar(X_train, y_clase_train)
    pred_clase = modelo_c.predecir(X_test)
    acc = accuracy_score(y_clase_test, pred_clase)
    print(f"Acierto sube/baja: {acc:.1%}")

    # TABLA DESCRIPTIVA
    vars_desc = ['cafe', 'ayer', 'ma5', 'ma20', 'volatilidad']
    nombres_bonitos = {
        'cafe': 'Precio café (USD/Lb)',
        'ayer': 'Ayer',
        'ma5': 'Prom. 5 días',
        'ma20': 'Prom. 20 días',
        'volatilidad': 'Volatilidad 20 días'
    }

    # Estadísticas básicas de las variables
    desc = datos.df[vars_desc].describe().loc[['mean','50%','std','min','max']].rename(index={'50%':'median'})
    desc = desc.round(4)

    # Renombra filas y agrega el MAE del test
    ren_idx = {'mean':'Media', 'median':'Mediana', 'std':'Desv. estándar', 'min':'Mínimo', 'max':'Máximo'}
    resumen = desc.rename(index=ren_idx).copy()
    resumen.loc['MAE (test)'] = np.nan
    resumen.loc['MAE (test)', 'cafe'] = round(mae, 4)

    # Crea la tabla visual con Plotly
    header_vals = ['Estadística'] + [nombres_bonitos[c] for c in vars_desc]
    cell_cols = [resumen.index.tolist()] + [resumen[c].tolist() for c in vars_desc]

    tabla = go.Figure(data=[go.Table(
        header=dict(values=header_vals, fill_color='#1f77b4', font_color='white', align='center'),
        cells=dict(values=cell_cols, fill_color='lavender', align='center')
    )])
    tabla.update_layout(title="Resumen descriptivo de variables")
    tabla.show()


    # GRAFICO DE SERIES CON HOVER (ÚLTIMOS 100 DÍAS)
    fechas = test_df.index.strftime('%Y-%m-%d').tolist()  # Fechas del test

    hover_text = []  # Crea texto para cada punto
    for i in range(len(y_precio_test)):
        fecha = fechas[i]
        real = y_precio_test[i]
        pred = pred_test_p[i]
        error = abs(real - pred)
        texto = (f"<b>Fecha:</b> {fecha}<br>"
                 f"<b>Real:</b> ${real:.2f}<br>"
                 f"<b>Pred:</b> ${pred:.2f}<br>"
                 f"<b>Error:</b> ${error:.2f}")
        hover_text.append(texto)

    # Serie real y predicha (últimos 100 días)
    data = [
        go.Scatter(
            x=fechas[-100:], y=y_precio_test[-100:],
            mode='lines', name='Precio Real',
            line=dict(color='blue'),
            hoverinfo='text', hovertext=hover_text[-100:]
        ),
        go.Scatter(
            x=fechas[-100:], y=pred_test_p[-100:],
            mode='lines', name='Predicción RF',
            line=dict(color='red', dash='dash'),
            hoverinfo='text', hovertext=hover_text[-100:]
        )
    ]

    layout = go.Layout(
        title="Predicción del Precio del Café (Hover para ver datos)",
        xaxis=dict(title="Fecha"),
        yaxis=dict(title="Precio (USD por Libra)"),
        hovermode='x unified'  # Muestra valores juntos al pasar el cursor
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

    # PREDICCIÓN PARA MAÑANA
    hoy = datos.df[['ayer', 'ma5', 'ma20', 'volatilidad', 'rsi']].iloc[-1:].values
    mañana = modelo_p.predecir(hoy)[0]         # Predicción de precio
    sube = modelo_c.predecir(hoy)[0]           # Predicción si sube o baja

    # Mostrar resultados en consola
    print(f"\nHOY: ${datos.df['cafe'].iloc[-1]:.2f} USD/Lbs")
    print(f"MAÑANA: ${mañana:.2f} USD/Lbs")
    print(f"SVM: {'SUBE' if sube else 'BAJA'}")


Bajando datos reales del cafe (KC=F)...
Creando features...
Listo: 486 filas
Entrenando Random Forest...


/tmp/ipython-input-2712719245.py:26: FutureWarning:

YF.download() has changed argument auto_adjust default to True



Error en test: 15.3 USD/Lbs
Entrenando SVM...
Acierto sube/baja: 50.5%



HOY: $413.60 USD/Lbs
MAÑANA: $399.19 USD/Lbs
SVM: BAJA
